In [8]:
import sys
print(sys.version)

3.12.1 (main, Aug 20 2024, 19:28:58) [GCC 9.4.0]


In [10]:
import folium

ModuleNotFoundError: No module named 'folium'

In [9]:
!pip install folium

In [ ]:
/opt/conda/envs/my_env/lib/python3.13/site-packages